In [19]:
import pandas as pd
import catboost 
import sklearn 
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool, cv
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna

In [20]:
data = pd.read_csv('../data/processed/cleaned_data.csv', sep=',')
X = data.drop('Preis', axis=1)
y = data['Preis']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [22]:
cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# model = CatBoostRegressor(iterations=10,
#                           learning_rate=1,
#                           depth=2)
# model.fit(X_train, y_train, cat_features=cat_features)
# preds = model.predict(X_test)
# mse = mean_squared_error(y_test, preds)
# r2 = r2_score(y_test, preds)
# mean_absolute_error = mean_absolute_error(y_test, preds)

# print("Mean Absolute Error:", mean_absolute_error)
# print("Mean Squared Error:", mse)
# print("R2 Score:", r2)

train_pool = Pool(X_train, y_train, cat_features=cat_features)

def objective(trial):
    params = {
        "depth": trial.suggest_int("depth", 6, 8),
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.3),
        "iterations": trial.suggest_int("iterations", 200, 800),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "random_strength": trial.suggest_float("random_strength", 0, 5),
        "loss_function": "RMSE",
        "early_stopping_rounds": 50,
        "verbose": False
    }

    cv_results = cv(
            pool=train_pool,
            params=params,
            fold_count=5,                 
            shuffle=True,
            partition_random_seed=42,
            verbose=False
        )
    best_rmse = np.min(cv_results["test-RMSE-mean"])
    return best_rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Beste Hyperparameter:", study.best_params)

best_model = CatBoostRegressor(**study.best_params, silent=True)
best_model.fit(X_train, y_train, cat_features=cat_features)

preds_log = best_model.predict(X_test)
preds = np.expm1(preds_log)
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mean_absolute_error = mean_absolute_error(y_test, preds)

print("Mean Absolute Error:", mean_absolute_error)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)
feature = best_model.get_feature_importance(prettified=True)
print(feature)


[I 2025-09-26 12:12:34,281] A new study created in memory with name: no-name-f9e07a9a-fabd-41d9-bb48-f44a6f4f7844


Training on fold [0/5]

bestTest = 0.4695179825
bestIteration = 62

Training on fold [1/5]

bestTest = 0.2700751006
bestIteration = 77

Training on fold [2/5]

bestTest = 0.3502084314
bestIteration = 424

Training on fold [3/5]

bestTest = 0.4348081727
bestIteration = 233

Training on fold [4/5]


[I 2025-09-26 12:12:37,595] Trial 0 finished with value: 0.412444633189348 and parameters: {'depth': 8, 'learning_rate': 0.2775863007901099, 'iterations': 431, 'l2_leaf_reg': 1.0751347153667408, 'bagging_temperature': 0.4424462880760526, 'random_strength': 3.2514267066256464}. Best is trial 0 with value: 0.412444633189348.



bestTest = 0.5336370528
bestIteration = 45

Training on fold [0/5]

bestTest = 0.403363772
bestIteration = 126

Training on fold [1/5]

bestTest = 0.2508807414
bestIteration = 106

Training on fold [2/5]

bestTest = 0.520657125
bestIteration = 86

Training on fold [3/5]

bestTest = 0.4702016838
bestIteration = 286

Training on fold [4/5]


[I 2025-09-26 12:12:41,251] Trial 1 finished with value: 0.38500319346258144 and parameters: {'depth': 7, 'learning_rate': 0.19132296785435476, 'iterations': 736, 'l2_leaf_reg': 4.6872194441918404, 'bagging_temperature': 0.6803568247028177, 'random_strength': 0.09170322546548093}. Best is trial 1 with value: 0.38500319346258144.


...


bestTest = 0.321430167
bestIteration = 106

Training on fold [0/5]

bestTest = 0.3100457158
bestIteration = 587

Training on fold [1/5]

bestTest = 0.2785154571
bestIteration = 301

Training on fold [2/5]

bestTest = 0.2748891753
bestIteration = 424

Training on fold [3/5]

bestTest = 0.4280260115
bestIteration = 343

Training on fold [4/5]


[I 2025-09-26 12:15:49,906] Trial 48 finished with value: 0.30114146335587394 and parameters: {'depth': 6, 'learning_rate': 0.1390592473882619, 'iterations': 792, 'l2_leaf_reg': 3.475357868954318, 'bagging_temperature': 0.2577619528931353, 'random_strength': 1.3681841522286664}. Best is trial 48 with value: 0.30114146335587394.



bestTest = 0.2096419151
bestIteration = 172

Training on fold [0/5]

bestTest = 0.4051944621
bestIteration = 573

Training on fold [1/5]

bestTest = 0.2477928295
bestIteration = 521

Training on fold [2/5]

bestTest = 0.3262208745
bestIteration = 257

Training on fold [3/5]

bestTest = 0.4211773898
bestIteration = 217

Training on fold [4/5]


[I 2025-09-26 12:15:53,248] Trial 49 finished with value: 0.32470783758361543 and parameters: {'depth': 6, 'learning_rate': 0.13657570697535715, 'iterations': 797, 'l2_leaf_reg': 2.2865925936915406, 'bagging_temperature': 0.261208484061704, 'random_strength': 1.5544538012495743}. Best is trial 48 with value: 0.30114146335587394.



bestTest = 0.2206637688
bestIteration = 145

Beste Hyperparameter: {'depth': 6, 'learning_rate': 0.1390592473882619, 'iterations': 792, 'l2_leaf_reg': 3.475357868954318, 'bagging_temperature': 0.2577619528931353, 'random_strength': 1.3681841522286664}
Mean Absolute Error: 8.821703886590079
Mean Squared Error: 272.4815298801845
R2 Score: -323.9937398988595
                              Feature Id  Importances
0                   Bauteilgewicht in kg    29.833490
1                                Flaeche    25.134759
2          Breite Laserzuschnitt in mm 2     9.286204
3          Breite Laserzuschnitt in mm 1     7.942556
4                                Bereich     7.357522
5                                 Dichte     5.580152
6   Gesamtlänge aller Aussparungen in mm     4.531557
7                 Anzahl der Abkantungen     3.973111
8                    Materialsärke in mm     3.038723
9                               Material     2.806696
10                          Bezugsquelle     0.